# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

## Open Connection

In [ ]:
import os
import re
import glob
import uuid
import psycopg2
import pandas as pd
import sql_queries as sq

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
conn = psycopg2.connect(
    user="admin",
    password="<password>",
    host="localhost", 
    port="5432",
    dbname="sparkifydb"
)
cur = conn.cursor()
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

def single_quote_converter(sentence):
    sentence = re.sub(r'\'', r'<single_quote_tag>', sentence)

    return sentence
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#

## Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
song_files = get_files(os.path.join(os.getcwd(), "data/song_data"))

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
print(f"Total number of JSON files: {len(song_files)}")
print(f"Example index 0: {os.path.basename(song_files[0])}")

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
filepath = song_files[0]

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
songs_df = pd.read_json(filepath, lines=True)
songs_df.head()

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
print(f"Artist ID, characters lenght: {len(songs_df['artist_id'].values[0])}")
print(f"Artist ID, characters lenght: {len(songs_df['song_id'].values[0])}")

### Load all `song_data`

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
songs_df = pd.DataFrame(columns=songs_df.columns)
for filepath in song_files:
    songs_df = songs_df.append(
        pd.read_json(filepath, lines=True),
        ignore_index=True
    )

print(f"Total number of songs:\n{songs_df.shape[0]}\n")
songs_df.head()

### `artists` Table

#### Extract Data for Artists Table

- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
columns = [
    "artist_id",
    "artist_name",
    "artist_location",
    "artist_latitude",
    "artist_longitude"
]
artist_data = songs_df[columns]
artist_data[:]["artist_name"] = artist_data["artist_name"].map(
    single_quote_converter, na_action='ignore'
)
artist_data[:]["artist_location"] = artist_data["artist_location"].map(
    single_quote_converter, na_action='ignore'
)
artist_data = artist_data.drop_duplicates()
artist_data = artist_data.reset_index(drop=True)
print(f"Total number of artists:\n{artist_data.shape[0]}\n")
artist_data.head()
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#

##### Insert Record into Artist Table

Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
# Test query creation
print("Print a query example (incomplete data, just a sample).\n")
query = sq.artist_table_insert(dataframe=artist_data[0:5], verbose=True)

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
start_index = 0
end_index = artist_data.shape[0]
batch_size = 5_000
for index in range(start_index, end_index, batch_size):
    print(f"Send batch from idx '{index}'...")
    query = sq.artist_table_insert(
        dataframe=artist_data.iloc[index:index + batch_size]
    )
    cur.execute(query)
    conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

### `songs` Table

#### Extract Data for Songs Table

- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
columns = ["song_id", "title", "artist_id", "year", "duration"]
song_data = songs_df[columns]
song_data[:]["title"] = song_data["title"].map(
    single_quote_converter, na_action='ignore'
)
song_data = song_data.drop_duplicates()
song_data = song_data.reset_index(drop=True)
print(f"Total number of songs:\n{song_data.shape[0]}\n")
song_data.head()

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
# Test query creation
query = sq.song_table_insert(dataframe=song_data.iloc[0:5], verbose=True)

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
start_index = 0
end_index = song_data.shape[0]
batch_size = 5_000
for index in range(start_index, end_index, batch_size):
    print(f"Send batch from idx '{index}'...")
    query = sq.song_table_insert(
        dataframe=song_data.iloc[index:index + batch_size]
    )
    cur.execute(query)
    conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
log_files = get_files(os.path.join(os.getcwd(), "data/log_data"))

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
print(f"Total number of JSON files: {len(log_files)}")
print(f"Example index 0: {os.path.basename(log_files[0])}")

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
filepath = log_files[0]

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
logs_df = pd.read_json(filepath, lines=True)
print(f"Total number of logs: {logs_df.shape[0]}")
logs_df.head()

In [ ]:
logs_df.info()

In [ ]:
logs_df["page"].value_counts()

### Load all `log_data`

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
logs_df = pd.DataFrame(columns=logs_df.columns)
for filepath in log_files:
    logs_df = logs_df.append(
        pd.read_json(filepath, lines=True),
        ignore_index=True
    )

print(f"Total number of logs:\n{logs_df.shape[0]}\n")
logs_df.head()

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
filtered_logs_df = logs_df[logs_df["page"] == 'NextSong'].copy()
del logs_df
print(f"Total number of logs with useful data:\n{filtered_logs_df.shape[0]}\n")
filtered_logs_df.head()

In [ ]:
filtered_logs_df.isnull().sum()

### `time` Table

#### Extract Data for Time Table

- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
filtered_logs_df['ts'] = pd.to_datetime(filtered_logs_df['ts'], unit='ms')
filtered_logs_df = filtered_logs_df.drop_duplicates()
print(f"Total number of logs with useful data:\n{filtered_logs_df.shape[0]}\n")
filtered_logs_df.head()

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
time_data = list(
    map(
        lambda item: [
            item.strftime('%Y-%m-%dT%H:%M:%S'),
            item.hour,
            item.day,
            item.week,
            item.month,
            item.year,
            item.weekday() + 1
        ],
        filtered_logs_df['ts']
    )
)
print(f"Print element list sample:\n{time_data[0]}\n")
column_labels = [
    'start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday'
]
print(f"Print column names:\n{column_labels}\n")
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
time_list_dict = [dict(zip(column_labels, row)) for row in time_data]
print(f"Print first row dictionary sample:\n{time_list_dict[0]}\n")

In [ ]:
time_df = pd.DataFrame(time_list_dict)
print(f"Current dataframe dimensions:\n{time_df.shape}\n")
time_df.head()

In [ ]:
time_df = pd.DataFrame(time_list_dict)
time_df = time_df.drop_duplicates()
time_df = time_df.reset_index(drop=True)
print(f"Current dataframe dimensions:\n{time_df.shape}\n")
time_df.head()

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
# Test query creation
query = sq.time_table_insert(dataframe=time_df.iloc[0:5], verbose=True)

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
start_index = 0
end_index = time_df.shape[0]
batch_size = 5_000
for index in range(start_index, end_index, batch_size):
    print(f"Send batch from idx '{index}'...")
    query = sq.time_table_insert(
        dataframe=time_df.iloc[index:index + batch_size]
    )
    cur.execute(query)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

### `users` Table

#### Extract Data for Users Table

- Select columns for user ID, first name, last name, gender and level and set to `user_df`
- Drop duplicates by ID but keep last user found, this stores the current User level status
- Keep in mind that the fact table stores the current level status at hearing the song, but
  then the user can change its level.

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
columns = ["userId", "firstName", "lastName", "gender", "level"]
user_df = filtered_logs_df[columns].copy()
user_df[:]["firstName"] = user_df["firstName"].map(
    single_quote_converter, na_action='ignore'
)
user_df[:]["lastName"] = user_df["lastName"].map(
    single_quote_converter, na_action='ignore'
)
user_df[:]["gender"] = user_df["gender"].str.upper()
user_df[:]["userId"] = user_df["userId"].astype(str)
user_df = user_df.drop_duplicates()
user_df = user_df.drop_duplicates(subset='userId', keep='last')
user_df = user_df.reset_index(drop=True)
print(f"Total number of users:\n{user_df.shape}\n")
user_df.head()

#### Insert Records into Users Table

Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
# Test query creation
query = sq.user_table_insert(dataframe=user_df.iloc[0:5], verbose=True)

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
start_index = 0
end_index = user_df.shape[0]
batch_size = 5_000
for index in range(start_index, end_index, batch_size):
    print(f"Send batch from idx '{index}'...")
    query = sq.user_table_insert(
        dataframe=user_df.iloc[index:index + batch_size]
    )
    cur.execute(query)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

### `songplays` Table

#### Extract Data and Songplays Table

This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

> **length** in log files is the song duration in seconds

#### Update `artists` and `songs` tables

<!--
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
-->
During processing current project, missing artists and songs were identified.

The log files contains artists and songs that are not registered in their
respective tables `songs` and `artists`. Before perform the inclution data
from the songplays, the missing songs and artists are included in the database,
creating a predefined system to get own song and artist IDs.


<!--
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
-->

##### Get songs and artists from logs

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
songs_columns = ["song_id", "title", "artist_id", "artist_name", "duration"]
all_songs_df = songs_df[songs_columns].copy()
all_songs_df = all_songs_df.drop_duplicates()
all_songs_df = all_songs_df.reset_index(drop=True)
print(f"Number of songs:\n{all_songs_df.shape[0]}\n")
all_songs_df.head()

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
logs_columns = ["song", "artist", "length"]
all_logs_df = filtered_logs_df[logs_columns].copy()
all_logs_df = all_logs_df.drop_duplicates()
all_logs_df = all_logs_df.reset_index(drop=True)
print(f"Number of logs:\n{all_logs_df.shape[0]}\n")
all_logs_df.head()

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
print(f"{len(set(all_logs_df['artist'].tolist()))} different artists.")
print(f"{len(set(all_logs_df['song'].tolist()))} different songs.")

##### Get missed artists

Artists including duplicates.

In [ ]:
artist_df = all_logs_df[['artist']]
print(f"Number of artists:\n{artist_df.shape[0]}\n")

Current artists in full songs.

In [ ]:
len([artist for artist in artist_df['artist'].tolist() if artist in all_songs_df['artist_name'].tolist()])

Artists no registered in full songs.


In [ ]:
len([artist for artist in artist_df['artist'].tolist() if artist not in all_songs_df['artist_name'].tolist()])

Artists no duplicates.

In [ ]:
artist_df = artist_df.drop_duplicates()
artist_df = artist_df.reset_index(drop=True)
print(f"Number of artists:\n{artist_df.shape[0]}\n")

Artists real number registered in full songs.

In [ ]:
len([artist for artist in artist_df['artist'].tolist() if artist in all_songs_df['artist_name'].tolist()])

Artists real number no registered in full songs.

In [ ]:
len([artist for artist in artist_df['artist'].tolist() if artist not in all_songs_df['artist_name'].tolist()])

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
# Keep only artists no located in 'all_songs_df'
artist_df = artist_df[
    ~artist_df['artist'].isin(all_songs_df['artist_name'])
].copy()
print(f"Number of artists:\n{artist_df.shape[0]}\n")
artist_df.head()

##### Assign UUID to artists

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
artist_df.insert(
    loc=0,
    column='artist_id',
    value=list(
        map(
            lambda item: uuid.uuid3(uuid.NAMESPACE_URL, item).hex[:18],
            artist_df['artist']
        )
    ),
    allow_duplicates=True
)
artist_df.head()

In [ ]:
artist_df['artist'].is_unique

In [ ]:
artist_df['artist_id'].is_unique

##### Generate complete artists dataframe

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
series_nan = nan_list = ['nan']*artist_df.shape[0]
artist_df.insert(
    loc=artist_df.shape[1],
    column='location',
    value=series_nan,
    allow_duplicates=True
)
artist_df.insert(
    loc=artist_df.shape[1],
    column='latitude',
    value=series_nan,
    allow_duplicates=True
)
artist_df.insert(
    loc=artist_df.shape[1],
    column='longitude',
    value=series_nan,
    allow_duplicates=True
)
artist_df[:]["artist"] = artist_df["artist"].map(
    single_quote_converter, na_action='ignore'
)
print(f"Number of artists:\n{artist_df.shape[0]}\n")
artist_df.head()

##### Upload missed artist data to `artists` table

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
start_index = 0
end_index = artist_df.shape[0]
batch_size = 5_000
for index in range(start_index, end_index, batch_size):
    print(f"Send batch from index '{index}'...")
    query = sq.artist_table_insert(
        dataframe=artist_df.iloc[index:index + batch_size]
    )
    cur.execute(query)
    conn.commit()

##### Get missed songs

In [ ]:
len([item for item in list(all_logs_df['song']) if item not in list(all_songs_df['title'])])

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
# Keep only songs no located in 'all_songs_df'
song_df = all_logs_df[
    ~all_logs_df[['song', 'artist', 'length']].isin(all_songs_df[['title', 'artist_name', 'duration']])
].copy()
song_df = song_df.drop_duplicates()
song_df = song_df.reset_index(drop=True)
print(f"Number of rows:\n{song_df.shape[0]}\n")
song_df.head()

##### Assign UUID to songs

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
song_df.insert(
    loc=0,
    column='song_id',
    value=list(
        map(
            lambda item: uuid.uuid3(uuid.NAMESPACE_URL, item).hex[:18],
            song_df[['song', 'length']].apply(
                lambda row: row[0]+str(row[1]),
                axis=1
            )
        )
    ),
    allow_duplicates=True
)
song_df[:]["song"] = song_df["song"].map(
    single_quote_converter, na_action='ignore'
)
song_df.head()

##### Get artist id

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
song_df['artist'] = song_df['artist'].map(
    lambda item: uuid.uuid3(uuid.NAMESPACE_URL, item).hex[:18]
)
song_df.head()

##### Generate complete songs dataframe

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
series_nan = nan_list = ['nan']*song_df.shape[0]
song_df.insert(
    loc=3,
    column='year',
    value=series_nan,
    allow_duplicates=True
)
song_df = song_df.drop_duplicates()
song_df.head()

##### Upload missed songs data to `songs` table

<!--
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
-->
The new UUIDs in songs and artists are in lower case to diferentiate between
the IDs from `song_data` and the IDs generated locally. On this step the new
songs and artists are uploaded to the `songs` and `artists` tables.

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
start_index = 0
end_index = song_df.shape[0]
batch_size = 5_000
for index in range(start_index, end_index, batch_size):
    print(f"Send batch from idx '{index}'...")
    query = sq.song_table_insert(
        dataframe=song_df.iloc[index:index + batch_size]
    )
    cur.execute(query)
    conn.commit()

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

##### Build base dataframe

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
columns = [
    "ts",
    "userId",
    "level",
    "song",
    "artist",
    "sessionId",
    "location",
    "userAgent"
]
song_plays_df = filtered_logs_df[columns].copy()

song_plays_df["ts"] = song_plays_df["ts"].dt.strftime('%Y-%m-%dT%H:%M:%S')
song_plays_df["userId"] = song_plays_df["userId"].astype(str)
song_plays_df["sessionId"] = song_plays_df["sessionId"].astype(str)
song_plays_df = song_plays_df.drop_duplicates()
song_plays_df = song_plays_df.reset_index(drop=True)
song_plays_df["location"] = song_plays_df["location"].map(
    single_quote_converter, na_action='ignore'
)
song_plays_df["userAgent"] = song_plays_df["userAgent"].map(
    single_quote_converter, na_action='ignore'
)
song_plays_df["song"] = song_plays_df["song"].map(
    single_quote_converter, na_action='ignore'
)
song_plays_df["artist"] = song_plays_df["artist"].map(
    single_quote_converter, na_action='ignore'
)
print(f"Total number of played songs:\n{song_plays_df.shape[0]}\n")
song_plays_df.head()
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#

##### Load corresponding IDs

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
# Test query
query = sq.song_select(
    dataframe=song_plays_df[["song", "artist"]].iloc[:2],
    verbose=True
)

In [ ]:
query = sq.song_select(
        dataframe=song_plays_df[columns].iloc[index:index + batch_size]
    )

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
start_index = 0
end_index = song_plays_df.shape[0]
batch_size = 5_000
columns = ["song", "artist"]
for index in range(start_index, end_index, batch_size):
    print(f"Get batch from index '{index}'...")
    query = sq.song_select(
        dataframe=song_plays_df[columns].iloc[index:index + batch_size]
    )
    temp_df = pd.read_sql(query, conn)
    song_plays_df.loc[index:index+batch_size, 'song'] = temp_df['song_id']
    song_plays_df.loc[index:index+batch_size, 'artist'] = temp_df['artist_id']

song_plays_df.head()

##### Upload songplays

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
# Test query creation
query = sq.songplay_table_insert(
    dataframe=song_plays_df.iloc[0:1],
    verbose=True
)

In [ ]:
song_plays_df.shape

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
start_index = 0
end_index = song_plays_df.shape[0]
batch_size = 5_000
for index in range(start_index, end_index, batch_size):
    print(f"Send batch from idx '{index}'...")
    query = sq.songplay_table_insert(
        dataframe=song_plays_df.iloc[index:index+batch_size]
    )
    cur.execute(query)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## Close Connection to Sparkify Database

In [ ]:
conn.close()

> Implement `etl.py`
> Use what you've completed in this notebook to implement `etl.py`.